In [1]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import auth

auth.authenticate_user()
from googleapiclient.discovery import build



files_to_load = list()

files_to_load.append({
    "file_name": "all_articles.json",
    "id": "19ErkUdKHwJO46T3u_LnoUhU-Ol4Om90W",
    "is_binary": 0
})


drive_service = build('drive', 'v3')


def upload_from_drive(file_id):
    import io
    from googleapiclient.http import MediaIoBaseDownload

    request = drive_service.files().get_media(fileId=file_id)
    downloaded = io.BytesIO()
    downloader = MediaIoBaseDownload(downloaded, request)
    done = False
    while done is False:
        # _ is a placeholder for a progress object that we ignore.
        # (Our file is small, so we skip reporting progress.)
        _, done = downloader.next_chunk()

    downloaded.seek(0)
    read = downloaded.read()
    print("Data loaded")
    return read


for file in files_to_load:
    print(file)

    # load document
    doc = upload_from_drive(file["id"])

    text_file = str()
    if file["is_binary"]:
        text_file = open(file["file_name"], "wb")

    else:
        doc = doc.decode("utf-8")
        text_file = open(file["file_name"], "w")

    text_file.write(doc)
    text_file.close()

    print("loaded: " + file["file_name"])

     |████████████████████████████████| 993kB 4.4MB/s 
{'file_name': 'all_articles.json', 'id': '19ErkUdKHwJO46T3u_LnoUhU-Ol4Om90W', 'is_binary': 0}
Data loaded
loaded: all_articles.json


In [2]:
!pip install gensim

Doc2vec training using only keywords extracted from text

In [3]:
"""
Single file script for google collab training
"""

import numpy as np
import re as re
import gensim
from gensim.models import doc2vec
from collections import namedtuple
# for stopwords
import nltk

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
import nltk

import random
from random import shuffle
random.seed(9001)

from gensim.test.utils import get_tmpfile
import pickle


import string
def substitute_contraction(word):
    """
    Substitutes the contraction with extended form
    Substites non-acsii qoutes
    :param word:
    :return:
        The substitued contraction, or the original token
    """

    # Replace unicode commas
    punctuation = {0x2018: 0x27, 0x2019: 0x27, 0x201C: 0x22, 0x201D: 0x22}
    w = word.translate(punctuation)

    contractions = get_contraction_dict()

    if w in contractions.keys():
        subbed = contractions[w]
        return subbed
    else:
        return w


def get_contraction_dict():
    return {
        "ain't": "is not",
        "aren't": "are not",
        "can't": "cannot",
        "can't've": "cannot have",
        "'cause": "because",
        "could've": "could have",
        "couldn't": "could not",
        "couldn't've": "could not have",
        "didn't": "did not",
        "doesn't": "does not",
        "don't": "do not",
        "hadn't": "had not",
        "hadn't've": "had not have",
        "hasn't": "has not",
        "haven't": "have not",
        "he'd": "he would",
        "he'd've": "he would have",
        "he'll": "he will",
        "he'll've": "he will have",
        "he's": "he has",
        "how'd": "how did",
        "how'd'y": "how do you",
        "how'll": "how will",
        "how's": "how is",
        "I'd": "I would",
        "I'd've": "I would have",
        "I'll": "I will",
        "I'll've": "I shall have",
        "I'm": "I am",
        "I've": "I have",
        "isn't": "is not",
        "it'd": "it would",
        "it'd've": "it would have",
        "it'll": "it will",
        "it'll've": "it shall have",
        "it's": "it is",
        "let's": "let us",
        "ma'am": "madam",
        "mayn't": "may not",
        "might've": "might have",
        "mightn't": "might not",
        "mightn't've": "might not have",
        "must've": "must have",
        "mustn't": "must not",
        "mustn't've": "must not have",
        "needn't": "need not",
        "needn't've": "need not have",
        "o'clock": "of the clock",
        "oughtn't": "ought not",
        "oughtn't've": "ought not have",
        "shan't": "shall not",
        "sha'n't": "shall not",
        "shan't've": "shall not have",
        "she'd": "she would",
        "she'd've": "she would have",
        "she'll": "she will",
        "she'll've": "she will have",
        "she's": "she is",
        "should've": "should have",
        "shouldn't": "should not",
        "shouldn't've": "should not have",
        "so've": "so have",
        "so's": "so is",
        "that'd": "that had",
        "that'd've": "that would have",
        "that's": "that is",
        "there'd": "there would",
        "there'd've": "there would have",
        "there's": "there is",
        "they'd": "they would",
        "they'd've": "they would have",
        "they'll": "they will",
        "they'll've": "they will have",
        "they're": "they are",
        "they've": "they have",
        "to've": "to have",
        "wasn't": "was not",
        "we'd": "we would",
        "we'd've": "we would have",
        "we'll": "we will",
        "we'll've": "we will have",
        "we're": "we are",
        "we've": "we have",
        "weren't": "were not",
        "what'll": "what will",
        "what'll've": "what will have",
        "what're": "what are",
        "what's": "what is",
        "what've": "what have",
        "when's": "when is",
        "when've": "when have",
        "where'd": "where did",
        "where's": "where is",
        "where've": "where have",
        "who'll": "who will",
        "who'll've": "who will have",
        "who's": "who is",
        "who've": "who have",
        "why's": "why is",
        "why've": "why have",
        "will've": "will have",
        "won't": "will not",
        "won't've": "will not have",
        "would've": "would have",
        "wouldn't": "would not",
        "wouldn't've": "would not have",
        "y'all": "you all",
        "y'all'd": "you all would",
        "y'all'd've": "you all would have",
        "y'all're": "you all are",
        "y'all've": "you all have",
        "you'd": "you would",
        "you'd've": "you would have",
        "you'll": "you will",
        "you'll've": "you will have",
        "you're": "you are",
        "you've": "you have"
    }




from string import punctuation
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
class LanguageProcessor:
    def __init__(self):
        """
        All of the natural processing functionality
        """

        self.lemmatizer = WordNetLemmatizer()

        self.punctuation_list = [c for c in punctuation]

        self.STOPWORDS = [' .', "'", "' '", "''", "'.", "'ll", "'re", "'s", "'ve", '--', '->', '..', '...', '``', 'a', "a's",
                     'able',
                     'about', 'above', 'abst', 'accordance', 'according', 'accordingly', 'across', 'act', 'actually',
                     'added',
                     'adj', 'affected', 'affecting', 'affects', 'after', 'afterwards', 'again', 'against', 'ah',
                     "ain't", 'all',
                     'allow', 'allows', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'am',
                     'among',
                     'amongst', 'an', 'and', 'announce', 'another', 'any', 'anybody', 'anyhow', 'anymore', 'anyone',
                     'anything',
                     'anyway', 'anyways', 'anywhere', 'apart', 'apparently', 'appear', 'appreciate', 'appropriate',
                     'approximately', 'are', 'area', 'areas', 'aren', "aren't", 'arent', 'arise', 'around', 'as',
                     'aside',
                     'ask', 'asked', 'asking', 'asks', 'associated', 'at', 'auth', 'available', 'away', 'awfully', 'b',
                     'back',
                     'backed', 'backing', 'backs', 'be', 'became', 'because', 'become', 'becomes', 'becoming', 'been',
                     'before',
                     'beforehand', 'began', 'begin', 'beginning', 'beginnings', 'begins', 'behind', 'being', 'beings',
                     'believe', 'below', 'beside', 'besides', 'best', 'better', 'between', 'beyond', 'big', 'biol',
                     'both',
                     'brief', 'briefly', 'but', 'by', 'c', "c'mon", "c's", 'ca', 'came', 'campaign', 'can', "can't",
                     'cannot',
                     'cant', 'case', 'cases', 'cause', 'causes', 'certain', 'certainly', 'changes', 'clear', 'clearly',
                     'co',
                     'com', 'come', 'comes', 'concerning', 'consequently', 'consider', 'considering', 'contain',
                     'containing',
                     'contains', 'corresponding', 'could', "couldn't", 'couldnt', 'course', 'criticized', 'have',
                     'currently',
                     'd', 'date', 'definitely', 'described', 'despite', 'did', "didn't", 'differ', 'different',
                     'differently',
                     'do', 'does', "doesn't", 'doing', "don't", 'done', 'down', 'downed', 'downing', 'downs',
                     'downwards',
                     'due', 'during', 'e', 'each', 'early', 'ed', 'edu', 'effect', 'eg', 'eight', 'eighty', 'either',
                     'else',
                     'elsewhere', 'end', 'ended', 'ending', 'endorsed', 'ends', 'enough', 'entirely', 'especially',
                     'et',
                     'et-al', 'etc', 'even', 'evenly', 'ever', 'every', 'everybody', 'everyone', 'everything',
                     'everywhere',
                     'ex', 'exactly', 'example', 'except', 'f', 'face', 'faces', 'fact', 'facts', 'far', 'felt', 'few',
                     'ff',
                     'fifth', 'financial', 'find', 'finds', 'first', 'five', 'fix', 'followed', 'following', 'follows',
                     'for',
                     'former', 'formerly', 'forth', 'found', 'four', 'from', 'full', 'fully', 'further', 'furthered',
                     'furthering', 'furthermore', 'furthers', 'g', 'gave', 'general', 'generally', 'get', 'gets',
                     'getting',
                     'give', 'given', 'gives', 'giving', 'go', 'goes', 'going', 'gone', 'good', 'goods', 'got',
                     'gotten',
                     'great', 'greater', 'greatest', 'greetings', 'group', 'grouped', 'grouping', 'groups', 'h', 'had',
                     "hadn't", 'happens', 'hardly', 'has', "hasn't", 'have', "haven't", 'having', 'he', "he's", 'hed',
                     'hello',
                     'help', 'hence', 'her', 'here', "here's", 'hereafter', 'hereby', 'herein', 'heres', 'hereupon',
                     'hers',
                     'herself', 'hes', 'hi', 'hid', 'high', 'higher', 'highest', 'him', 'himself', 'his', 'hither',
                     'home',
                     'hopefully', 'how', 'howbeit', 'however', 'hundred', 'i', "i'd", "i'll", "i'm", "i've", 'id', 'ie',
                     'if',
                     'ignored', 'im', 'immediate', 'immediately', 'importance', 'important', 'in', 'inasmuch', 'inc',
                     'indeed',
                     'index', 'indicate', 'indicated', 'indicates', 'information', 'inner', 'insofar', 'instead',
                     'interest',
                     'interested', 'interesting', 'interests', 'into', 'invention', 'inward', 'is', "isn't", 'it',
                     "it'd",
                     "it'll", "it's", 'itd', 'its', 'itself', 'j', 'just', 'k', 'keep', 'keep', 'keeps', 'kept', 'kg',
                     'kind',
                     'km', 'knew', 'know', 'known', 'knows', 'l', 'large', 'largely', 'last', 'lately', 'later',
                     'latest',
                     'latter', 'latterly', 'least', 'less', 'lest', 'let', "let's", 'lets', 'like', 'liked', 'likely',
                     'line',
                     'little', 'long', 'longer', 'longest', 'look', 'looking', 'looks', 'ltd', 'm', 'made', 'mainly',
                     'make',
                     'makes', 'making', 'man', 'many', 'may', 'maybe', 'me', 'mean', 'means', 'meantime', 'meanwhile',
                     'member',
                     'members', 'men', 'merely', 'mg', 'might', 'million', 'miss', 'ml', 'more', 'moreover', 'most',
                     'mostly',
                     'mr', 'mr.', 'mrs', 'much', 'mug', 'must', 'my', 'myself', 'n', "n't", 'na', 'name', 'namely',
                     'nay', 'nd',
                     'near', 'nearly', 'necessarily', 'necessary', 'need', 'needed', 'needing', 'needs', 'neither',
                     'never',
                     'nevertheless', 'new', 'newer', 'newest', 'next', 'nine', 'ninety', 'no', 'nobody', 'non', 'none',
                     'nonetheless', 'noone', 'nor', 'normally', 'nos', 'not', 'noted', 'nothing', 'novel', 'now',
                     'nowhere',
                     'number', 'numbered', 'numbering', 'numbers', 'o', 'obtain', 'obtained', 'obviously', 'of', 'off',
                     'official', 'often', 'oh', 'ok', 'okay', 'old', 'older', 'oldest', 'omitted', 'on', 'once', 'one',
                     'ones',
                     'only', 'onto', 'open', 'opened', 'opening', 'opens', 'or', 'ord', 'order', 'ordered', 'ordering',
                     'orders', 'other', 'others', 'otherwise', 'ought', 'our', 'ours', 'ourselves', 'out', 'outside',
                     'over',
                     'overall', 'owing', 'own', 'p', 'page', 'pages', 'part', 'parted', 'particular', 'particularly',
                     'parting',
                     'parts', 'past', 'per', 'perhaps', 'place', 'placed', 'places', 'please', 'plus', 'point',
                     'pointed',
                     'pointing', 'points', 'poorly', 'possible', 'possibly', 'potentially', 'pp', 'predominantly',
                     'present',
                     'presented', 'presenting', 'presents', 'presumably', 'previously', 'primarily', 'probably',
                     'problem',
                     'problems', 'promptly', 'proud', 'provides', 'put', 'puts', 'q', 'quarterly', 'que', 'quickly',
                     'quite',
                     'quote', 'qv', 'r', 'ran', 'rather', 'rd', 're', 'readily', 'really', 'reasonably', 'recent',
                     'recently',
                     'ref', 'refs', 'regarding', 'regardless', 'regards', 'relatively', 'related', 'relatively',
                     'research',
                     'respectively', 'resulted', 'resulting', 'results', 'right', 'room', 'rooms', 'run', 's', 'said',
                     'same',
                     'saw', 'say', 'saying', 'says', 'sec', 'second', 'secondly', 'seconds', 'section', 'see', 'seeing',
                     'seem',
                     'seemed', 'seeming', 'seems', 'seen', 'sees', 'self', 'selves', 'sensible', 'sent', 'serious',
                     'seriously',
                     'seven', 'several', 'shall', 'sharply', 'she', "she'll", 'shed', 'shes', 'should', "shouldn't",
                     'show',
                     'showed', 'showing', 'shown', 'showns', 'shows', 'side', 'sides', 'significant', 'significantly',
                     'similar', 'similarly', 'since', 'six', 'slightly', 'small', 'smaller', 'smallest', 'so', 'some',
                     'somebody', 'somehow', 'someone', 'somethan', 'something', 'sometime', 'sometimes', 'somewhat',
                     'somewhere', 'soon', 'sorry', 'specifically', 'specified', 'specify', 'specifying', 'state',
                     'states',
                     'still', 'stop', 'strongly', 'sub', 'substantially', 'successfully', 'such', 'sufficiently',
                     'suggest',
                     'sup', 'sure', 'sure', 't', "t's", 'take', 'taken', 'taking', 'tell', 'tends', 'th', 'than',
                     'thank',
                     'thanks', 'thanx', 'that', "that'll", "that's", "that've", 'thats', 'the', 'their', 'theirs',
                     'them',
                     'themselves', 'then', 'thence', 'there', "there'll", "there's", "there've", 'thereafter',
                     'thereby',
                     'thered', 'therefore', 'therein', 'thereof', 'therere', 'theres', 'thereto', 'thereupon', 'these',
                     'they',
                     "they'd", "they'll", "they're", "they've", 'theyd', 'theyre', 'thing', 'things', 'think', 'thinks',
                     'third', 'this', 'thorough', 'thoroughly', 'those', 'thou', 'though', 'thoughh', 'thought',
                     'thoughts',
                     'thousand', 'three', 'throug', 'through', 'throughout', 'thru', 'thus', 'til', 'tip', 'to',
                     'today',
                     'together', 'too', 'took', 'toward', 'towards', 'tried', 'tries', 'truly', 'try', 'trying', 'ts',
                     'turn',
                     'turned', 'turning', 'turns', 'twice', 'two', 'u', 'un', 'under', 'unfortunately', 'unless',
                     'unlike',
                     'unlikely', 'until', 'unto', 'up', 'upon', 'ups', 'us', 'use', 'used', 'useful', 'usefully',
                     'usefulness',
                     'uses', 'using', 'usually', 'uucp', 'v', 'value', 'various', 'very', 'via', 'viz', 'vol', 'vols',
                     'vs',
                     'w', 'want', 'wanted', 'wanting', 'wants', 'was', "wasn't", 'wasnt', 'way', 'ways', 'we', "we'd",
                     "we'll",
                     "we're", "we've", 'wed', 'welcome', 'well', 'wells', 'went', 'were', "weren't", 'werent', 'what',
                     "what'll", "what's", 'whatever', 'whats', 'when', 'whence', 'whenever', 'where', "where's",
                     'whereafter',
                     'whereas', 'whereby', 'wherein', 'wheres', 'whereupon', 'wherever', 'whether', 'which', 'while',
                     'whim',
                     'whither', 'who', "who'll", "who's", 'whod', 'whoever', 'whole', 'whom', 'whomever', 'whos',
                     'whose',
                     'why', 'widely', 'will', 'willing', 'wish', 'with', 'within', 'without', "won't", 'wonder', 'wont',
                     'words', 'work', 'worked', 'working', 'works', 'world', 'would', "wouldn't", 'wouldnt', 'www', 'x',
                     'y',
                     'year', 'years', 'yes', 'yet', 'you', "you'd", "you'll", "you're", "you've", 'youd', 'young',
                     'younger',
                     'youngest', 'your', 'youre', 'yours', 'yourself', 'yourselves', 'z', 'zero', 'zerowhose', '’', '“',
                     '”',
                     'will', 'thing', "n't", "''", "'s", '``', "'re", "'", 'mr', 'mr.', '--', '...', '..', '->', "'.",
                     "' '",
                     ' .', '’', '“', '”', '', '\n']

        # TODO: Get more complete list and read it from file
        MORESTOP = ['will', 'thing', 'n\'t', '\'\'', '\'s', '``', '\'re', '\'', 'mr', 'mr.', '--', '...', '..', '->', '\'.',
                    '\' \'', ' .', '’',
                    '“', '”', "", "\n"]
        for m in MORESTOP:
            self.STOPWORDS.append(m)

    def substitute_contractions(self, text):
        """
        Loop through words and sub contractions
        :param text:
        :return:
        """
        subbed = []
        for word in text.split():
            subbed.append(substitute_contraction(word))
        return " ".join(subbed)

    def get_non_stopwords(self, text, substitute_contractions=True, stem=True):
        """
        Returns a list of lowercase non-stopwords in the text.
        non-stopwords are anything that is not punctuation or stopwords
        Numerical values are NOT FILTERED OUT
        :param text:
        :param stem:
        :return:
        """

        if substitute_contractions:
            subbed_text = self.substitute_contractions(text)
        else:
            subbed_text = text

        non_stop_words = []
        tokens = word_tokenize(subbed_text)

        # Loop through tokens
        for tok in tokens:
            t = tok.lower()
            token = self.remove_punctuation(t)
            if token not in self.STOPWORDS:
                # Check if token contains punctuation
                if token not in self.punctuation_list:
                    if stem:
                        non_stop_words.append(self.get_word_lemma(token))
                    else:
                        non_stop_words.append(token)

        return non_stop_words

    def get_noun_phrases(self, text, filter_stopwords=True):
        """
        Gets the noun phrases from a text, by parsing grammar POS tagged
        :param text:
        :param filter_stopwords"
            Filters the stopwords from noun-phrases
        :return:
            Returns a list of noun phrases with list for each scentence
        """

        sentences = nltk.sent_tokenize(text)
        sentences = [nltk.word_tokenize(sent) for sent in sentences]
        sentences = [nltk.pos_tag(sent) for sent in sentences]

        grammar = """NP: {<DT>?<JJ>*<NN.*>+}
                  """

        # Alternative grammer structures that can be parsed, can add to the grammar string
        """
        RELATION: {<V.*>}
                  {<DT>?<JJ>*<NN.*>+}
        ENTITY: {<NN.*>}
        """

        cp = nltk.RegexpParser(grammar)
        noun_phrases_list = [[' '.join(leaf[0] for leaf in tree.leaves())
                              for tree in cp.parse(sent).subtrees()
                              if tree.label() == 'NP']
                             for sent in sentences]

        if not filter_stopwords:
            return noun_phrases_list

        # Filters the stopwords from the parsed noun phrases
        else:
            phrases = []
            for elem in noun_phrases_list:
                for noun_phrase in elem:

                    # Search each word in return phrase to remove stopwords
                    phrase = []
                    for word in noun_phrase.split():
                        w = word.lower()
                        if w not in self.STOPWORDS:
                            phrase.append(w)

                    if phrase != []:
                        phrases.append(" ".join(phrase))

            return phrases

    # TODO: Improve parsing speed with spacy
    def get_adjectives_list(self, text):
        """
        Uses the POS tagger to get all of the Adjectives, whose tags start with "j"
        :param text:
        :return:
        """

        tokens = nltk.word_tokenize(text)

        adjectives = []
        tagged = nltk.pos_tag(tokens)
        for t in tagged:
            # Check the first charcter if tag
            if t[1][0] == "J":
                adjectives.append(t[0])

        return adjectives

    # TODO: Improve parsing speed with spacy
    def get_nouns_list(self, text):
        """
        Uses the POS tagger to get all of the Adjectives, whose tags start with "j"
        :param text:
        :return:
        """

        tokens = nltk.word_tokenize(text)

        nouns = []
        tagged = nltk.pos_tag(tokens)
        for t in tagged:
            # Check the first charcter if tag
            if t[1][0] == "N":
                nouns.append(t[0])

        return nouns

    def get_word_lemma(self, word):
        """
        Helper to allows customization to stemming process, like checking for trailing e's
        :param word:
        :return:
        """
        lema = self.lemmatizer.lemmatize(word)
        return lema

    def remove_punctuation(self, text):
        """
        Helper function to remove all non-acsii charcters
        :param text:
        :return:
        """
        return ''.join([i if ord(i) < 128 else '' for i in text])

    def is_text_token(self, token):
        """
        Checks if not punc or numerical, or non-acsii
        :param token:
        :return:
        """

        if len(token) == 1:
            if ord(token) < 128 and token not in punctuation and not token.isdigit():
                return True
            else:
                return False

        else:
            if not token.isdigit():
                return True
            else:
                return False

from nltk.tokenize import sent_tokenize
from nltk.probability import FreqDist

class SummaryException(Exception):
    """
    If text could not meet summary requirments, lenght, content...
    TODO: from summa import summarizer, summarizer performs well, graph based
    """
    pass


class Cleaner(LanguageProcessor):
    """
    Freqeucy based summarizer
    Summarizes to the min/max number of tokens required in summary
    Use .summarize

    Frequency based keyword list
    Highest frequency noun_phrases
    """

    def __init__(self, text, number_keywords=100):
        """

        :param text:
        :param title:
        :param include_intro:
            How many intro sentences to include in summary
        """

        LanguageProcessor.__init__(self)

        # Min/Max tokens in summary
        self.minimum_length = 0
        self.maximum_length = 1000
        
        self.minimum_tokens = 50

        # Tokens that indicate sentence should not be in summary
        self.non_body_sentence_blacklist = ["Twitter\n", "Facebook\n", "Instagram\n", "Getty Images", "\n"]

        self.text = text
        
        self.number_keywords = number_keywords

        # All non-stopwords tokens
        self.tokens = []

        # All non-stopwords tokens
        self.noun_phrases = []

        # Dict of scentences and tokens
        self.sentence_dict = {}

        # The intro sentences
        self.intro = []
        self.summarized_body = []

        # Used for frequency based methods
        self.tokens_freq = FreqDist()
        self.noun_phrase_freq = FreqDist()

        # Must parse first
        self.parsed = False

    def clean(self):
        """
        Performs NLP methods
        :return:
        """

        self.refined_text_dict()
        self.all_tokens()
        self.all_noun_phrases()

        # Used for frequency based methods
        self.tokens_freq = FreqDist(self.tokens)

        self.noun_phrase_freq = FreqDist(self.noun_phrases)

        # Saves the intro sentences, and removes from dict

        self.parsed = True

    def keywords(self):
        """
        Gets tokens from freqeucny distribution
        :param number_keywords:
        :return:
        """

        if not self.parsed:
            raise SummaryException("Must call summarize first!")

        keys = self.tokens_freq.most_common()

        # Loop to just get keywords not the counts
        keywords_list = []
        for word in keys:
            keywords_list.append(word[0])

        return keywords_list[:500]

    def key_noun_phrases(self, number_noun_phrases=5):
        """
        Gets tokens from freqeucny distribution
        :param number_noun_phrases:
        :return:
        """

        if not self.parsed:
            raise SummaryException("Must call summarize first!")

        keys = self.noun_phrase_freq.most_common()

        # Loop to just get keywords not the counts
        noun_phrases_list = []
        for word in keys:
            noun_phrases_list.append(word[0])

        return noun_phrases_list

    def refined_text_dict(self, stem=True):
        """
        Creates a dict of the scentences and the processed tokens in the sentences
        Removes stopwords and can stem
        :return:
            Sets a dict of tokens of the sentences and tokens
        """

        sentence_tokenized = sent_tokenize(self.text)

        # Initilize dict from keys, with [ ] as initial value
        scentence_dict = dict([(key, []) for key in sentence_tokenized])

        all_tokens = []

        # Simple loop to tokenize and add words to the sent dict, hashed by the words in sent
        for sent in sentence_tokenized:
            tokens = self.get_non_stopwords(sent, stem=stem)
            scentence_dict[sent] = tokens

        # Returns all tokens, used for Freq Distribution
        for scentence, scentence_tokens in scentence_dict.items():
            for tok in scentence_tokens:
                all_tokens.append(tok)


        self.sentence_dict = scentence_dict

    def all_tokens(self):

        # Get all tokens
        for scentence, scentence_tokens in self.sentence_dict.items():
            for tok in scentence_tokens:
                if self.is_text_token(tok):
                    self.tokens.append(tok)
                    
#         if len(self.tokens) < self.number_keywords:
#             raise SummaryException("Not enough text to summarize!")

    def all_noun_phrases(self):

        # Get noun phrases, ensure length greater than 1
        noun_phrases = self.get_noun_phrases(self.text)
        self.noun_phrases = [phrase for phrase in noun_phrases if len(phrase.split()) > 1]



file = "all_articles.json"
save_file = "health_doc2vec"

content = []

import json

with open(file) as f:
    content = json.load(f)

li_doc = [con["text"] for con in content]

li_doc = li_doc[:]

print("Train doc {0}".format(len(li_doc)))



def text_cleaning_for_embeding(text):
  """
  Formate the text for the document embedding
  :param x:
  :return:
  """

  text_cleaner = Cleaner(text, number_keywords=50)
  tokens = []

  try:
    text_cleaner.clean()
    keyword = text_cleaner.keywords()
    phrases = text_cleaner.key_noun_phrases(number_noun_phrases=10)
    phrases = [phrase.replace(" ", "_") for phrase in phrases]

    tokens = keyword + phrases
  except SummaryException:
    pass

  return tokens


# Transform data to 'TaggedDocument'
docs_train = []
analyzedDocument = namedtuple('AnalyzedDocument', 'words tags')
cachedStopWords = stopwords.words("english")


print("Preprocessing Text")
for i, text in enumerate(li_doc):
    if len(text.split()) > 250:
      
      try:
        processed_text = text_cleaning_for_embeding(text)
        # register as namedtuple: analyzedDocument
        tags = [i]

        #     if len(processed_text) > 0:
        docs_train.append(analyzedDocument(processed_text, tags))
      except TypeError:
        pass
      
print("Total cleaned: {0}".format(len(docs_train)))

print("Train gensim")
# dm=0 turns of the distibuted memory varient, just try to predict which words from which paragraph
model = gensim.models.Doc2Vec(dm=0, vector_size=250, window=10, min_count=2, workers=8, alpha=0.025, epochs=35)

# sample=1e-4, negative=5,
# shuffling is better (ot needed at each trianing epoch
shuffle(docs_train)
# Build vocabulary from a sequence of sentences
model.build_vocab(docs_train)
# Update the model’s neural weights from a sequence of sentences

print("Save gensim")
model.train(docs_train, epochs=model.epochs, total_examples=model.corpus_count)

with open(save_file, 'wb') as handle:
    pickle.dump(model, handle)

# testing on training sentences:
for i in range(0, 50):
    t = docs_train[i].tags[0]
    new_vector = model.infer_vector(docs_train[i].words, steps=50, alpha=0.025)
    sims = model.docvecs.most_similar(
        positive=[new_vector])  # gives you top 10 document tags and their cosine similarity
    print(sims)

    # model.docvecs.most_similar(): get similarity between word vector
    # model.most_similar(): get similarity between document
    print('--------------------------------------------------------------------------------------------------------')
    print('------------- %s' % li_doc[t].split()[:50])
    print('--------------------------------------------------------------------------------------------------------')
    for t, sim in sims:
        print('\n', sim, li_doc[t].split()[:50])
        
        
        
import json
model.delete_temporary_training_data(keep_doctags_vectors=False, keep_inference=True)
with open(save_file, 'wb') as handle:
    pickle.dump(model, handle)
handle.close()
    
import os
os.path.getsize(save_file)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
Train doc 18840
Preprocessing Text
Total cleaned: 16138
Train gensim
Save gensim


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[(4617, 0.9520883560180664), (4430, 0.46648910641670227), (4859, 0.46641334891319275), (15905, 0.46432942152023315), (11342, 0.4495343863964081), (17774, 0.4478479027748108), (10512, 0.4473556876182556), (8514, 0.4466628432273865), (2796, 0.44605398178100586), (8438, 0.4448506236076355)]
--------------------------------------------------------------------------------------------------------
------------- ['Microsoft', 'last', 'week', 'filed', 'its', 'annual', 'report', 'with', 'the', 'SEC,', 'and', 'with', 'it', 'a', 'new', 'vision', 'that', 'emphasizes', 'AI.', 'The', 'documents', 'state', 'the', 'company', 'will', 'no', 'longer', 'be', 'focused', 'on', 'mobile,', 'but', 'instead', 'on', 'implementing', 'AI', 'solutions.', 'In', 'the', 'documents,', 'under', 'the', 'heading', '“Our', 'Vision”', 'the', 'company', 'put:', 'Our', 'strategy']
--------------------------------------------------------------------------------------------------------

 0.9520883560180664 ['Microsoft', 'last', 

359850246

Doc2Vec using full tokenized text stopwords removed.

In [6]:


#FOR COLAB TRAINING:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

from google.colab import auth

auth.authenticate_user()

from googleapiclient.discovery import build

drive_service = build('drive', 'v3')


def save_doc_google(filename):
    from googleapiclient.http import MediaFileUpload

    file_metadata = {
        'name': filename,
        'mimeType': 'text/plain'
    }
    media = MediaFileUpload(filename,
                            mimetype='text/plain',
                            resumable=True)
    created = drive_service.files().create(body=file_metadata,
                                           media_body=media,
                                           fields='id').execute()
    print('File ID: {}'.format(created.get('id')))

    
save_doc_google("health_doc2vec")

KeyboardInterrupt: ignored

In [0]:
import numpy as np
import re as re
import gensim
from gensim.models import doc2vec
from collections import namedtuple
#for stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import nltk

import random
from random import shuffle
random.seed(9001)

from gensim.test.utils import get_tmpfile
import pickle

file = "all_articles.json"
save_file = "my_doc2vec_model"
fname = get_tmpfile("my_doc2vec_model")

content = []

import json
with open(file) as f:
    content = json.load(f)
    
li_doc = [con["text"] for con in content]

print("Train doc {0}".format(len(li_doc)))


def text_cleaning_for_embeding(x):
    #remove ponctuation
    x = ' '.join(re.split('[?.,:/>!;]', x))
    #remove entre parenthèse
    x = x.replace('(','')
    x = x.replace(')','')
    #remove digits
    x = ''.join([l.lower() for l in x if not l.isdigit()])
    #remove stopwords and tokenize
    x = ' '.join([w for w in x.split() if w not in cachedStopWords])
    return(x)

#Transform data to 'TaggedDocument'
docs = []
analyzedDocument = namedtuple('AnalyzedDocument', 'words tags')
cachedStopWords = stopwords.words("english")

for i, text in enumerate(li_doc):
    text_ = text_cleaning_for_embeding(text)
    #list of words
    words = text_.split()
    #small cleaning
    words = [i.lower().strip() for i in words]
    #register as namedtuple: analyzedDocument
    tags = [i]
    docs.append(analyzedDocument(words, tags))

docs_train = docs[:2000]
print(len(docs_train))
# docs_test = docs[500:len(docs)]

print("Train gensim")
model = gensim.models.Doc2Vec(dm=0, vector_size=100, window=10, min_count=1, workers=8, alpha=0.025, min_alpha=0.015, 
                              epochs=20)
#sample=1e-4, negative=5,
#shuffling is better (ot needed at each trianing epoch
shuffle(docs_train)
#Build vocabulary from a sequence of sentences 
model.build_vocab(docs_train)
#Update the model’s neural weights from a sequence of sentences

print("Train gensim")
with open(save_file, 'wb') as handle:
  pickle.dump(model, handle)
model.train(docs_train, epochs=model.epochs, total_examples=model.corpus_count)
with open(save_file, 'wb') as handle:
  pickle.dump(model, handle)

#testing on 2 training sentences:
for i in range(0,50):
    t = docs_train[i].tags[0]
    new_vector = model.infer_vector(docs_train[i].words[:125],steps=50, alpha=0.025)
    sims = model.docvecs.most_similar(positive=[new_vector]) #gives you top 10 document tags and their cosine similarity
    print(sims)
    
    #model.docvecs.most_similar(): get similarity between word vector
    #model.most_similar(): get similarity between document
    print('--------------------------------------------------------------------------------------------------------')
    print('------------- %s'%li_doc[t].split()[:20])
    print('--------------------------------------------------------------------------------------------------------')
    for t,sim in sims:
        print('\n', sim, li_doc[t].split()[:20])